# Overview

A client wants to purchase a home in Westchester county and wants to ensure their home is a good investment. They measure that by ROI they will get in 5 years. Their budget is $500K. Our firm is asked to recommend 5 zipcodes in Westchester county that would be good options.

# Business Understanding

- Objective: Identify five ZIP codes in Westchester County where a client can purchase a home within a $500,000 budget, aiming for optimal Return on Investment (ROI) over a 10-year period.

- Stakeholder: A prospective homebuyer seeking a property in Westchester County that not only fits within their financial constraints but also promises substantial appreciation.

- Problem: Determining which ZIP codes in Westchester County offer homes priced at or below $500,000 and are projected to yield the highest ROI over a 5-year span.

Key Questions
1. Which ZIP codes in Westchester County have homes available within the $500,000 budget?
2. What are the historical and projected real estate trends in these ZIP codes?
3. Which of these areas are anticipated to provide the highest ROI over the next 10 years?

Success Metrics: ROI (Return on Investment) over a 5-year period.

# Data Understanding

Our data set contains historical home prices across the US from 1996 to 2018.
* RegionID
* RegionName
* City
* State	Metro
* CountyName
* SizeRank
* Dates and their median home prices

In [ ]:
# install packages
!pip install pmdarima
!pip install folium

In [ ]:
# import relevant packages
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.dates as mdates 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error

# statsmodels
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from sklearn.model_selection import TimeSeriesSplit
from pmdarima import auto_arima
import pmdarima as pm
import statsmodels.api as sm
import folium
%matplotlib inline

In [ ]:
# Load the data and preview data
df=pd.read_csv('zillow_data.csv')
df.head()

### Filtering for Westchester
Since our client is only interested in purchasing a house in Westchester county we will filter the data to only include those zipcodes.

In [ ]:
# Since we know we are only looking in Westchester county, let's filter our dataset for that

westchester_df = df[df['CountyName'] == 'Westchester']
westchester_df.head()

In [ ]:
df.describe()

In [ ]:
df.shape

In [ ]:
# Basic EDA: summary statistics, data types, and city distribution
print("Basic Statistics (non-date columns):")
non_date_columns = ['RegionID', 'RegionName', 'SizeRank']
print(westchester_df[non_date_columns].describe())

print("\nData Information:")
print(westchester_df.info())

print("\nTop 10 Cities by Frequency:")
print(westchester_df['City'].value_counts().head(10))


# Data Preparation

In [ ]:
# How many zip codes are in each city
zipcode_counts = westchester_df.groupby('City')['RegionName'].nunique().reset_index()
zipcode_counts.columns = ['City', 'Num_Zipcodes']
print(zipcode_counts)

### Filtering for Home Prices under $500K

In [ ]:
filtered_df = westchester_df[westchester_df['2018-04'] <=500_000]

# Display the filtered DataFrame
filtered_df.head(15)

In [ ]:
zipcode_counts = filtered_df.groupby('City')['RegionName'].nunique().reset_index()
zipcode_counts.columns = ['City', 'Num_Zipcodes']
print(zipcode_counts)

In [ ]:
# Check for null values
filtered_df.isna().sum()

In [ ]:
# Convert dates to datetime objects
def get_datetimes(df):
    """
    Takes a dataframe:
    Returns only those column names that can be converted into datetime objects
    as datetime objects.
    """
    date_columns = [col for col in df.columns if pd.to_datetime(col, format='%Y-%m', errors='coerce') is not pd.NaT]
    return pd.to_datetime(date_columns, format='%Y-%m')

# Run the function
datetime_columns = get_datetimes(filtered_df)
print(datetime_columns)

In [ ]:
# rename columns
filtered_df = filtered_df.rename(columns={'RegionName': 'Zipcode'})
filtered_df

### Visualizing Home Prices over time in each zipcode

In [ ]:
# Step 1: Reshape the DataFrame
melted_df = pd.melt(filtered_df, id_vars=['Zipcode', 'City', 'State', 'Metro', 'CountyName'], 
                    var_name='time', value_name='value')

# Step 2: Convert 'time' to datetime
melted_df['time'] = pd.to_datetime(melted_df['time'], errors='coerce')

# Step 3: Plot the data
plt.figure(figsize=(10, 6))
for zipcode, group in melted_df.groupby('Zipcode'):
    plt.plot(group['time'], group['value'], label=str(zipcode))

plt.xlabel('Time')
plt.ylabel('Home Price')
plt.title('Home Prices Over Time by ZIP Code')
plt.legend(title='ZIP Code', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

### Filter out dates before 2009
Due to the housing market crash in 2008, all of our zipcodes saw a drastic dip during that time and the following years. To filter out some of that noise we will filter out all dates and prices before 2009.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Step 1: Reshape the DataFrame
melted_df = pd.melt(filtered_df, 
                    id_vars=['Zipcode', 'City', 'State', 'Metro', 'CountyName'], 
                    var_name='time', 
                    value_name='value')

# Step 2: Convert 'time' to datetime
melted_df['time'] = pd.to_datetime(melted_df['time'], errors='coerce')

# Step 2a: Filter for dates after 2008 (i.e., from 2009 onward)
melted_df = melted_df[melted_df['time'] >= pd.to_datetime('2009-01-01')]

# Step 3: Plot the data
plt.figure(figsize=(10, 6))
for zipcode, group in melted_df.groupby('Zipcode'):
    plt.plot(group['time'], group['value'], label=str(zipcode))

plt.xlabel('Time')
plt.ylabel('Home Price')
plt.title('Home Prices Over Time by ZIP Code (After 2008)')
plt.legend(title='ZIP Code', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
# Calculate ROI
pivot_df = melted_df.pivot_table(index='Zipcode', columns='time', values='value')
pivot_df['ROI'] = (pivot_df.iloc[:, -1] - pivot_df.iloc[:, 0]) / pivot_df.iloc[:, 0] * 100

plt.figure(figsize=(10, 6))
pivot_df['ROI'].sort_values().plot(kind='barh', color='skyblue')
plt.xlabel('Return on Investment (%)')
plt.ylabel('ZIP Code')
plt.title('ROI by ZIP Code')
plt.tight_layout()
plt.show()

### Reshape data from wide to long format (melt function)

In [ ]:
def melt_data(df):
    # Select only the date columns for melting
    date_columns = [col for col in df.columns if isinstance(col, str) and '-' in col]
    melted = pd.melt(df, id_vars=['Zipcode', 'City', 'State', 'Metro', 'CountyName'], value_vars=date_columns, var_name='time', value_name='HomePrice')
    
    # Convert the 'time' column to datetime
    melted['time'] = pd.to_datetime(melted['time'], errors='coerce')
    
    # Drop rows where 'time' could not be converted
    melted = melted.dropna(subset=['time'])
    
    # Group by 'time' and calculate the mean 'HomePrice'
    result = melted.groupby('time')['HomePrice'].mean().reset_index()
    
    return result

### Choosing a zipcode to create our model
We willuse 10704 as an example zip code to create our model.

In [ ]:
# Filter for 10704
filtered_df_10704=filtered_df[filtered_df['Zipcode']==10704]

In [ ]:
ts_10704=melt_data(filtered_df_10704)
ts_10704

In [ ]:
# Assuming your DataFrame is named df
ts_10704['time'] = pd.to_datetime(ts_10704['time'], errors='coerce')

# Filter for dates after 2008 (starting from January 1, 2009)
ts_10704 = ts_10704[ts_10704['time'] >= pd.to_datetime('2009-01-01')]

# Set the 'time' column as the index
ts_10704.set_index('time', inplace=True)

In [ ]:
ts_10704

### Visualize home prices in zipcode 10704

In [ ]:
# visualize home prices in 10704
plt.figure(figsize=(10, 6))
plt.plot(ts_10704)
plt.title('Real Estate Prices in Zipcode 10704 Over Time')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
ts_10704

## Arima Modeling

Split data into training and test data and visualize breakdown.

In [ ]:
# Split data into train and test
train_size = int(len(ts_10704) * 0.8)
train, test = ts_10704.iloc[:train_size], ts_10704.iloc[train_size:]

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(train.index, train, label='Train')
plt.plot(test.index, test, label='Test')
plt.title('Train-Test Split')
plt.legend()
plt.show()

In [ ]:
# Time Series Split on train
split=TimeSeriesSplit()

for train_ind, val_ind in split.split(train):
    print(f'Train Index: {train_ind}')
    print(f'Test Index: {val_ind}')

# Auto Arima

In [ ]:
auto_model = pm.auto_arima(
    train,         
    start_p=0,             # Starting value of p (AR order)
    start_q=0,             # Starting value of q (MA order)
    test='adf',            # Unit root test to check stationarity (ADF test)
    max_p=5,               # Maximum value of p
    max_q=5,               # Maximum value of q
    m=1,                   # Number of periods in each season (1 for non-seasonal data)
    d=0,                   # Order of non-seasonal differencing
    seasonal=True,         # Whether to include seasonal components
    start_P=0,             # Starting value of P (seasonal AR order)
    start_Q=0,             # Starting value of Q (seasonal MA order)
    D=0,                   # Order of seasonal differencing
    trace=True,            # Whether to print progress messages
    error_action='ignore', # Action to take when an error occurs ('ignore' to suppress)
    suppress_warnings=True # Whether to suppress warnings
)

Best model:  ARIMA(2,0,3)

In [ ]:
arima_model = sm.tsa.statespace.SARIMAX(train, 
                                        order=(4,0,2), 
                                        seasonal_order=(0, 0, 0, 0), 
                                        enforce_stationarity=False, 
                                        enforce_invertibility=False)

# Fit the model and print results
output = arima_model.fit()

print(output.summary().tables[1])
auto_model.plot_diagnostics(figsize=(18,18))
plt.show()

### Forecasting

In [ ]:
forecast_steps = 60  # 5 years * 12 months/year

In [ ]:
forecast = output.get_forecast(steps=forecast_steps)
forecast_mean = forecast.predicted_mean
forecast_conf_int = forecast.conf_int()

In [ ]:
forecast_dates = pd.date_range(start=train.index[-1] + pd.Timedelta(days=1), periods=forecast_steps, freq='M')

In [ ]:
# Plot historical data
plt.plot(train.index, train, label='Historical Data', color='blue')

# Plot forecasts
plt.plot(forecast_dates, forecast_mean, label='10-Year Forecast', color='red', linestyle='--')

# Plot confidence intervals
plt.fill_between(forecast_dates, forecast_conf_int.iloc[:, 0], forecast_conf_int.iloc[:, 1], color='red', alpha=0.2)

# Add labels and legend
plt.xlabel('Date')
plt.ylabel('Home Price')
plt.title('5-Year Forecast of Home Prices with Confidence Intervals')
plt.legend()

# Display the plot
plt.show()

## Applying model to the other Westchester zip codes

In [ ]:
filtered_df

In [ ]:
# Create an array of only the Westchester zip codes
westchester_zipcodes = filtered_df['Zipcode'].unique()
westchester_zipcodes

In [ ]:
# Initialize an empty dictionary to store filtered DataFrames
filtered_dfs = {}

# Iterate over each unique ZIP code
for zip_code in westchester_zipcodes:
    # Filter the DataFrame for the current ZIP code
    filtered_dfs[zip_code] = filtered_df[filtered_df['Zipcode'] == zip_code]

In [ ]:
# Initialize an empty dictionary to store the melted DataFrames
melted_dfs = {}

# Iterate over each unique ZIP code
for zipcode in westchester_zipcodes:
    # Filter the DataFrame for the current ZIP code
    filtered_df_zipcode = filtered_df[filtered_df['Zipcode'] == zipcode]
    
    # Apply the melt_data function
    melted_df = melt_data(filtered_df_zipcode)
    
    # Convert 'time' to datetime and set it as the index
    melted_df['time'] = pd.to_datetime(melted_df['time'], errors='coerce')
    melted_df = melted_df[melted_df['time'] >= pd.to_datetime('2009-01-01')]
    melted_df.set_index('time', inplace=True)
    
    # Store the melted DataFrame in the dictionary with the ZIP code as the key
    melted_dfs[f'ts_{zipcode}'] = melted_df

In [ ]:
melted_dfs

### Apply AutoARIMA to all zipcodes to Forecast future home prices
Now that we have a time series for each zip code, we will apply auto ARIMA to each of them to find the optimal parameters to forecast home prices 5 for the next 5 years.

In [ ]:
# Initialize a dictionary to store forecasted data for plotting
forecasted_data = {}

# Initialize a dictionary to store ROI for each ZIP code
zip_predictions = {}

# Initialize a list to store the final results for the DataFrame
data_list = []

# Iterate over each key-value pair in the dictionary
for zipcode, ts in melted_dfs.items():
    # Ensure the DataFrame is sorted by time index
    ts = ts.sort_index()
    
    # Split data into train and test sets
    train_size = int(len(ts) * 0.8)
    train, test = ts.iloc[:train_size], ts.iloc[train_size:]
    
    # Auto ARIMA model on the training set
    auto_model = pm.auto_arima(train['HomePrice'], start_p=0, start_q=0,
                               test='adf', max_p=5, max_q=5, m=1, d=0,
                               seasonal=True, start_P=0, start_Q=0, D=0,
                               trace=True, error_action='ignore',
                               suppress_warnings=True, stepwise=True,
                               with_intercept=False)
   
    # Fit the SARIMAX model on the entire series
    arima_model = sm.tsa.statespace.SARIMAX(ts['HomePrice'], 
                                            order=auto_model.order, 
                                            seasonal_order=auto_model.seasonal_order, 
                                            enforce_stationarity=False, 
                                            enforce_invertibility=False)

    # Fit the model
    output = arima_model.fit()
    
    # Forecast for the next 60 periods
    forecast = output.get_forecast(steps=60)
    prediction = forecast.conf_int()
    prediction['prediction'] = forecast.predicted_mean
    prediction.columns = ['lower', 'upper', 'prediction']
    
    # Store forecasted data for plotting
    forecast_dates = pd.date_range(start=ts.index[-1] + pd.Timedelta(days=1), periods=60, freq='MS')
    forecast_df = pd.DataFrame({'HomePrice': prediction['prediction'].values}, index=forecast_dates)
    forecasted_data[zipcode] = forecast_df
    
    # Calculate ROI and add to the zip_predictions dictionary
    last_actual = ts['HomePrice'].iloc[-1]
    last_predicted = prediction['prediction'].iloc[-1]
    roi = (last_predicted - last_actual) / last_actual
    zip_predictions[zipcode] = roi
    
    # Add the data to the list for the DataFrame
    data_list.append([zipcode, last_actual, last_predicted, roi])

# Create a DataFrame from the list
df = pd.DataFrame(data_list, columns=['Zipcode', 'Last_Actual_Price', 'Last_Predicted_Price', 'ROI'])

# Display the DataFrame
print(df)

In [ ]:
# Remove 'ts_' prefix from 'Zipcode' column
df['Zipcode'] = df['Zipcode'].str.removeprefix('ts_')
df

### Visualize forecasts and ROI for each zipcode

In [ ]:
# Plot all forecasts on the same graph
plt.figure(figsize=(10, 6))
for zipcode, forecast_df in forecasted_data.items():
    plt.plot(forecast_df.index, forecast_df['HomePrice'], label=f'ZIP Code {zipcode}')

plt.xlabel('Time')
plt.ylabel('Home Price')
plt.title('Home Price Forecasts for ZIP Codes')
plt.legend(title='ZIP Code', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# Plot ROI for each ZIP code
plt.figure(figsize=(10, 6))
plt.bar(zip_predictions.keys(), zip_predictions.values(), color='skyblue')
plt.xlabel('ZIP Code')
plt.ylabel('ROI')
plt.title('ROI for Each ZIP Code in Westchester')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Visualize home prices for top 5 zipcodes by ROI

In [ ]:
# Sort the ZIP codes by ROI in descending order and select the top 5
top_5_zipcodes = sorted(zip_predictions, key=zip_predictions.get, reverse=True)[:5]

# Plot the forecasts for the top 5 ZIP codes
plt.figure(figsize=(10, 6))
for zipcode in top_5_zipcodes:
    forecast_df = forecasted_data[zipcode]
    plt.plot(forecast_df.index, forecast_df['HomePrice'], label=f'ZIP Code {zipcode}')

plt.xlabel('Time')
plt.ylabel('Home Price')
plt.title('Home Price Forecasts for Top 5 ZIP Codes by ROI')
plt.legend(title='ZIP Code', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
top_5_df = df.sort_values(by='ROI', ascending=False).head(5)
top_5_df

### Plot ROI for top 5 zip codes

In [ ]:
# Plotting
plt.figure(figsize=(10, 6))
bars = plt.bar(top_5_df['Zipcode'].astype(str), top_5_df['ROI'], color=colors, edgecolor='black', linewidth=1.2)

# Add labels and title
plt.xlabel('ZIP Code', fontsize=12, fontweight='bold')
plt.ylabel('ROI', fontsize=12, fontweight='bold')
plt.title('Top 5 ZIP Codes by ROI', fontsize=14, fontweight='bold', pad=15)

# Customize grid lines
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Remove top and right spines
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Display ROI values above bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height + 0.005, f'{height:.2%}',
             ha='center', va='bottom', fontsize=12, fontweight='bold', color='black')

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
# Merge the DataFrames on 'Zipcode'
merged_df = pd.merge(top_5_df, filtered_df, on='Zipcode', how='left')

final_df = merged_df[['Zipcode', 'City', 'State', 'Metro', 'CountyName', 'SizeRank',
                       'Last_Actual_Price', 'Last_Predicted_Price', 'ROI']]

# Display the final DataFrame
final_df

## Recommendations

The top zipcodes by expected ROI over 5 years are:
1. 10701 - Yonkers
2. 10704 - Yonkers
3. 10598 - Yorktown
4. 10705 - Yonkers
5. 10703 - Yonkers

In [ ]:
# Approximate latitude and longitude for each ZIP code
zip_coords = {
    '10701': [40.931, -73.890],
    '10705': [40.946, -73.865],
    '10704': [40.946, -73.902],
    '10598': [41.305, -73.866],
    '10703': [40.975, -73.880]}

In [ ]:
# Create a base map centered roughly on Westchester County
m = folium.Map(location=[40.93, -73.90], zoom_start=11)

# Add markers for each ZIP code in the merged DataFrame
for idx, row in merged_df.iterrows():
    zipcode = row['Zipcode']
    lat, lon = zip_coords.get(zipcode, [40.93, -73.90])
    popup_text = f"{row['City']}, {row['State']} (Zip: {zipcode})<br>ROI: {row['ROI']:.2%}"
    folium.Marker(
        location=[lat, lon],
        popup=popup_text,
        icon=folium.Icon(color='blue', icon='home', prefix='fa')
    ).add_to(m)

# Save the map to an HTML file and display it
m.save('top_zipcodes_map.html')
m

# Conclusion

Based on the time series analysis and ARIMA modeling, we identified ZIP codes in Westchester County with homes priced at or below $500K and projected their home prices over the next 5 years. By calculating the expected ROI for each ZIP code, we can now highlight the top 5 areas that present the most promising investment opportunities for our client

## Next Steps